In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices() 

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15870554199573971419, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 11332668621
 locality {
   bus_id: 2
 }
 incarnation: 17261517923572989447
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:84:00.0"]

In [2]:
import numpy as np
import pandas as pd

from scipy import ndimage
from scipy import misc

import os, gc, sys, glob
from tqdm import tqdm

from sklearn import model_selection
from sklearn import metrics

import keras
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Input, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D

from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [5]:
DATA_DIR = os.path.join('/scratch', 'yns207', 'data_invasive')

path = DATA_DIR
test_path = os.path.join(path, 'test')
models_path = os.path.join(path, 'results')
train_path = os.path.join(path, 'train')
valid_path = os.path.join(path, 'valid')
print(path)

/scratch/yns207/data_invasive


In [12]:
%cd $DATA_DIR
!module load centos/7
!7za x '*.7z'

/scratch/yns207/data_invasive

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=fr_FR.UTF-8,Utf16=on,HugeFiles=on,64 bits,28 CPUs Intel(R) Xeon(R) CPU E5-2690 v4 @ 2.60GHz (406F1),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        2 files, 3353865207 bytes (3199 MiB)
    
Extracting archive: test.7z
--
Path = test.7z
Type = 7z
Physical Size = 1227586386
Headers Size = 16731
Method = LZMA2:24
Solid = +
Blocks = 1

      0% 1 - test/1.jp                    0% 7 - test/1003.j                      0% 17 - test/1012.jp                        0% 23 - test/1018.jp                        0% 32 - test/1026.jp                        0% 37 - test/1030.jp                        1% 44 - test/1037.jp                        1% 52 - test/1044.jp                        1% 63 - test/1054.jp                        1% 68 - test/1059.jp                        1% 73 - test/1063.jp                        1% 78 - test/1068.jp                       

 3   36%     36% 4 - train/1000.jp                       36% 8 - train/1004.jp                       37% 13 - train/1009.j                       37% 17 - train/1012.j                       37% 21 - train/1016.j                       37% 25 - train/102.jp                       37% 29 - train/1023.j                       37% 33 - train/1027.j                       37% 37 - train/1030.j                       37% 41 - train/1034.j                       37% 45 - train/1038.j                       38% 49 - train/1041.j                       38% 53 - train/1045.j                       38% 58 - train/105.jp                       38% 62 - train/1053.j                       38% 67 - train/1058.j                       38% 72 - train/1062.j                       38% 75 - train/1065.j                       38% 80 - train/107.jp                       39% 84 - train/1073.j                       39% 87 - train/1076.j                       39% 93 - train/1081.j                       39% 96 - train/1084

                         56% 731 - train/1656.jp                         56% 735 - train/166.j                       57% 739 - train/1663.jp                         57% 744 - train/1668.jp                         57% 748 - train/1671.jp                         57% 752 - train/1675.jp                         57% 756 - train/1679.jp                         57% 759 - train/1681.jp                         57% 764 - train/1686.jp                         57% 768 - train/169.j                       58% 772 - train/1693.jp                         58% 777 - train/1698.jp                         58% 782 - train/1701.jp                         58% 786 - train/1705.jp                         58% 790 - train/1709.jp                         58% 795 - train/1713.jp                         58% 798 - train/1716.jp                         58% 802 - train/172.j                       58% 806 - train/1723.jp                         59% 810 - train/1727.jp                         59% 814 - train/1730.jp   

                         76% 1442 - train/23.j                       76% 1445 - train/232.jp                         76% 1449 - train/236.jp                         76% 1453 - train/24.j                       76% 1458 - train/244.jp                         77% 1462 - train/248.jp                         77% 1466 - train/251.jp                         77% 1470 - train/255.jp                         77% 1475 - train/26.j                       77% 1479 - train/263.jp                         77% 1483 - train/267.jp                         77% 1488 - train/271.jp                         77% 1491 - train/274.jp                         78% 1495 - train/278.jp                         78% 1499 - train/281.jp                         78% 1504 - train/286.jp                         78% 1508 - train/29.j                       78% 1512 - train/293.jp                         78% 1516 - train/297.jp                         78% 1520 - train/30.j                       78% 1521 - train/300.jp            

                         95% 2141 - train/86.j                       95% 2145 - train/863.jp                         96% 2149 - train/867.jp                         96% 2153 - train/870.jp                         96% 2158 - train/875.jp                         96% 2162 - train/879.jp                         96% 2167 - train/883.jp                         96% 2171 - train/887.jp                         96% 2175 - train/890.jp                         96% 2179 - train/894.jp                         96% 2180 - train/895.jp                         97% 2183 - train/898.jp                         97% 2188 - train/901.jp                         97% 2191 - train/904.jp                         97% 2195 - train/908.jp                         97% 2199 - train/911.jp                         97% 2204 - train/916.jp                         97% 2208 - train/92.j                       97% 2211 - train/922.jp                         97% 2213 - train/924.jp                         97% 2218 - train/929.j

In [6]:
train_set = pd.read_csv(os.path.join(path, 'train_labels.csv'))
test_set = pd.read_csv(os.path.join(path, 'sample_submission.csv'))

In [7]:
train_set.head(2)

,name,invasive
0,1,0
1,2,0


In [8]:
def read_img(img_path):
    img = misc.imread(img_path)
    img = misc.imresize(img, (128, 128))
    return img

In [9]:
train_img, test_img = [],[]
for img_path in tqdm(train_set['name'].iloc[:]):
    train_img.append(read_img(os.path.join(path, 'train', str(img_path)+'.jpg')))

for img_path in tqdm(test_set['name'].iloc[:]):
    test_img.append(read_img(os.path.join(path, 'test', str(img_path)+'.jpg')))

100%|██████████| 1531/1531 [00:47<00:00, 32.01it/s]


In [10]:
train_img = np.array(train_img, np.float32)/255
test_img = np.array(test_img, np.float32)/255

In [11]:
train_label = np.array(train_set['invasive'].iloc[:])

In [12]:
def conv_block(filter_depth, filter_size, pool_size, activation, inputs):
    x = Convolution2D(filter_depth, filter_size, activation=activation)(inputs)
    x = MaxPooling2D(pool_size=pool_size)(x)
    return x

In [13]:
def dense_block(units, activation, drop_prob, inputs):
    x = Dense(units, activation=activation)(inputs)
    x = Dropout(drop_prob)(x)
    return x

In [14]:
def output_block(units, activation, inputs):
    x = Dense(units, activation=activation)(inputs)
    return x

In [27]:
def grab_optimizer(opt, lr):
    if opt == 'sgd':
        return optimizers.SGD(lr=lr, decay=1e-6, momentum=0.8, nesterov=True)
    elif opt == 'adam':
        return optimizers.Adam(lr=lr)
    elif opt == 'adagrad':
        return optimizers.Adagrad(lr=lr)
    elif opt == 'rmsprop':
        return optimizers.RMSprop(lr=lr)

In [25]:
def make_model(input_shape, optimizer):
    inputs = Input(shape=input_shape)
    m = conv_block(16, (3,3), (2,2),'relu', inputs=inputs)
    m = conv_block(32, (3,3), (2,2), 'relu', inputs=m)
    m = conv_block(64, (3,3), (2,2), 'relu', inputs=m)
    m = conv_block(128, (3,3), (2,2), 'relu', inputs=m)
    m = Flatten()(m)
    m = dense_block(2048, 'relu', 0.65, inputs=m)
    m = dense_block(512, 'relu', 0.55, inputs=m)
    outputs = dense_block(1, 'sigmoid', 0, inputs=m)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [17]:
# try a few rounds of training
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(train_img, train_label, test_size=0.20)

In [18]:
model = make_model((128,128,3), grab_optimizer('sgd', 0.01))

early_stopping = EarlyStopping(monitor='val_loss', patience=7, verbose=1, mode='auto')

gen = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest')

# only required of featurewise center or zca whitening or a few other things
# gen.fit(x_train)

hist = model.fit_generator(gen.flow(x_train, y_train, batch_size=64),
                    steps_per_epoch=(len(x_train)//64) + 1,
                    validation_data=(x_valid,y_valid),
                    validation_steps=(len(x_valid)//64)+1,
                    epochs=25,
                    verbose=2,
                    callbacks=[early_stopping])

Epoch 1/25
5s - loss: 0.6598 - acc: 0.6137 - val_loss: 0.6654 - val_acc: 0.6035
Epoch 2/25
4s - loss: 0.6360 - acc: 0.6380 - val_loss: 0.6375 - val_acc: 0.6035
Epoch 3/25
4s - loss: 0.6099 - acc: 0.6389 - val_loss: 0.5997 - val_acc: 0.6035
Epoch 4/25
4s - loss: 0.5834 - acc: 0.6577 - val_loss: 0.5709 - val_acc: 0.6623
Epoch 5/25
4s - loss: 0.5509 - acc: 0.7110 - val_loss: 0.5315 - val_acc: 0.7342
Epoch 6/25
4s - loss: 0.5231 - acc: 0.7369 - val_loss: 0.5161 - val_acc: 0.7386
Epoch 7/25
4s - loss: 0.5239 - acc: 0.7304 - val_loss: 0.4946 - val_acc: 0.7560
Epoch 8/25
4s - loss: 0.4880 - acc: 0.7704 - val_loss: 0.4556 - val_acc: 0.8126
Epoch 9/25
4s - loss: 0.4396 - acc: 0.7898 - val_loss: 0.6404 - val_acc: 0.6797
Epoch 10/25
4s - loss: 0.4397 - acc: 0.7941 - val_loss: 0.4874 - val_acc: 0.7495
Epoch 11/25
4s - loss: 0.4212 - acc: 0.8051 - val_loss: 0.3876 - val_acc: 0.8039
Epoch 12/25
4s - loss: 0.3538 - acc: 0.8417 - val_loss: 0.3149 - val_acc: 0.8758
Epoch 13/25
4s - loss: 0.3327 - acc: 

In [45]:
# ok now lets try a k folds

early_stopping = EarlyStopping(monitor='val_loss', patience=7, verbose=1, mode='auto')

kf = model_selection.KFold(n_splits=10, shuffle=True)
score_func = metrics.roc_auc_score

i = 0
va_score_per_model = {i: 0 for i in range(10)}

for rain_ixs, valid_ixs in kf.split(train_img):
    x_train = train_img[train_ixs]
    x_valid = train_img[valid_ixs]
    y_train = train_label[train_ixs]
    y_valid = train_label[valid_ixs]

    gen = ImageDataGenerator(
        rotation_range = 30,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        vertical_flip = True,
        fill_mode = 'nearest')

    # only required of featurewise center or zca whitening or a few other things
    # gen.fit(x_train)

    model = make_model((128,128,3), grab_optimizer('sgd', 0.005))
    
    model_checkpoint = ModelCheckpoint('/scratch/yns207/data_invasive/model_jun21_kfold_'+str(i)+'.model', 
                                        monitor='val_loss', 
                                        save_best_only=True)
    
    print('kfold: {}'.format(str(i)))
    
    hist = model.fit_generator(gen.flow(x_train, y_train, batch_size=64),
                        steps_per_epoch=(len(x_train)//64) + 1,
                        validation_data=(x_valid,y_valid),
                        validation_steps=(len(x_valid)//64)+1,
                        epochs=50,
                        verbose=2,
                        callbacks=[early_stopping, model_checkpoint])
    
    tr_score = score_func(y_train, model.predict(x_train)[:, 0])
    va_score = score_func(y_valid, model.predict(x_valid)[:, 0])
    va_score_per_model[i] = va_score
    
    print('tr score: {}, va score: {}'.format(tr_score, va_score))
    print('\n')
    i += 1

kfold: 0
Epoch 1/50
5s - loss: 0.6699 - acc: 0.6078 - val_loss: 0.6548 - val_acc: 0.6174
Epoch 2/50
4s - loss: 0.6536 - acc: 0.6277 - val_loss: 0.6496 - val_acc: 0.6174
Epoch 3/50
5s - loss: 0.6455 - acc: 0.6353 - val_loss: 0.6436 - val_acc: 0.6174
Epoch 4/50
5s - loss: 0.6449 - acc: 0.6285 - val_loss: 0.6355 - val_acc: 0.6174
Epoch 5/50
5s - loss: 0.6320 - acc: 0.6301 - val_loss: 0.6229 - val_acc: 0.6174
Epoch 6/50
5s - loss: 0.6145 - acc: 0.6298 - val_loss: 0.5995 - val_acc: 0.6174
Epoch 7/50
5s - loss: 0.5930 - acc: 0.6344 - val_loss: 0.5661 - val_acc: 0.6261
Epoch 8/50
4s - loss: 0.5619 - acc: 0.6653 - val_loss: 0.6061 - val_acc: 0.6304
Epoch 9/50
4s - loss: 0.5438 - acc: 0.6959 - val_loss: 0.7257 - val_acc: 0.4217
Epoch 10/50
4s - loss: 0.6488 - acc: 0.6125 - val_loss: 0.6198 - val_acc: 0.6174
Epoch 11/50
5s - loss: 0.5918 - acc: 0.6370 - val_loss: 0.5631 - val_acc: 0.7957
Epoch 12/50
5s - loss: 0.5291 - acc: 0.7065 - val_loss: 0.4791 - val_acc: 0.7261
Epoch 13/50
4s - loss: 0.492

4s - loss: 0.6471 - acc: 0.6309 - val_loss: 0.6387 - val_acc: 0.6261
Epoch 3/50
5s - loss: 0.6403 - acc: 0.6340 - val_loss: 0.6304 - val_acc: 0.6261
Epoch 4/50
5s - loss: 0.6307 - acc: 0.6309 - val_loss: 0.6149 - val_acc: 0.6261
Epoch 5/50
5s - loss: 0.6151 - acc: 0.6311 - val_loss: 0.5871 - val_acc: 0.6261
Epoch 6/50
5s - loss: 0.5935 - acc: 0.6397 - val_loss: 0.5508 - val_acc: 0.6174
Epoch 7/50
5s - loss: 0.5621 - acc: 0.6711 - val_loss: 0.5198 - val_acc: 0.6652
Epoch 8/50
4s - loss: 0.5435 - acc: 0.7099 - val_loss: 0.5920 - val_acc: 0.6391
Epoch 9/50
4s - loss: 0.5185 - acc: 0.7396 - val_loss: 0.5252 - val_acc: 0.6696
Epoch 10/50
5s - loss: 0.4980 - acc: 0.7560 - val_loss: 0.4484 - val_acc: 0.7609
Epoch 11/50
5s - loss: 0.4852 - acc: 0.7562 - val_loss: 0.3979 - val_acc: 0.8565
Epoch 12/50
5s - loss: 0.4711 - acc: 0.7786 - val_loss: 0.3761 - val_acc: 0.8304
Epoch 13/50
5s - loss: 0.4475 - acc: 0.7936 - val_loss: 0.3604 - val_acc: 0.8348
Epoch 14/50
4s - loss: 0.4152 - acc: 0.8123 - v

5s - loss: 0.6579 - acc: 0.6132 - val_loss: 0.6514 - val_acc: 0.6043
Epoch 2/50
4s - loss: 0.6387 - acc: 0.6309 - val_loss: 0.6382 - val_acc: 0.6043
Epoch 3/50
5s - loss: 0.6300 - acc: 0.6285 - val_loss: 0.6197 - val_acc: 0.6043
Epoch 4/50
5s - loss: 0.6060 - acc: 0.6314 - val_loss: 0.5915 - val_acc: 0.6043
Epoch 5/50
5s - loss: 0.5733 - acc: 0.6374 - val_loss: 0.5560 - val_acc: 0.6870
Epoch 6/50
5s - loss: 0.5471 - acc: 0.6847 - val_loss: 0.5218 - val_acc: 0.7043
Epoch 7/50
5s - loss: 0.5113 - acc: 0.7410 - val_loss: 0.5056 - val_acc: 0.6913
Epoch 8/50
5s - loss: 0.4997 - acc: 0.7542 - val_loss: 0.4721 - val_acc: 0.8304
Epoch 9/50
4s - loss: 0.4871 - acc: 0.7576 - val_loss: 0.5301 - val_acc: 0.7348
Epoch 10/50
5s - loss: 0.4663 - acc: 0.7871 - val_loss: 0.4640 - val_acc: 0.7957
Epoch 11/50
5s - loss: 0.4404 - acc: 0.7942 - val_loss: 0.4027 - val_acc: 0.8087
Epoch 12/50
4s - loss: 0.4199 - acc: 0.8072 - val_loss: 0.4199 - val_acc: 0.8087
Epoch 13/50
5s - loss: 0.4024 - acc: 0.8156 - va

tr score: 0.9748936727709626, va score: 0.9712069887680513


kfold: 6
Epoch 1/50
5s - loss: 0.6583 - acc: 0.6102 - val_loss: 0.6357 - val_acc: 0.6419
Epoch 2/50
4s - loss: 0.6422 - acc: 0.6296 - val_loss: 0.6241 - val_acc: 0.6419
Epoch 3/50
5s - loss: 0.6349 - acc: 0.6277 - val_loss: 0.6091 - val_acc: 0.6419
Epoch 4/50
5s - loss: 0.6205 - acc: 0.6301 - val_loss: 0.5871 - val_acc: 0.6419
Epoch 5/50
5s - loss: 0.5974 - acc: 0.6306 - val_loss: 0.5518 - val_acc: 0.6376
Epoch 6/50
5s - loss: 0.5707 - acc: 0.6458 - val_loss: 0.5190 - val_acc: 0.6550
Epoch 7/50
4s - loss: 0.5482 - acc: 0.6826 - val_loss: 0.6398 - val_acc: 0.6332
Epoch 8/50
5s - loss: 0.5352 - acc: 0.7148 - val_loss: 0.4905 - val_acc: 0.7860
Epoch 9/50
5s - loss: 0.5137 - acc: 0.7380 - val_loss: 0.4612 - val_acc: 0.7860
Epoch 10/50
5s - loss: 0.4914 - acc: 0.7410 - val_loss: 0.4258 - val_acc: 0.8079
Epoch 11/50
5s - loss: 0.4853 - acc: 0.7597 - val_loss: 0.4182 - val_acc: 0.7991
Epoch 12/50
4s - loss: 0.4673 - acc: 0.7797 - va

tr score: 0.9804174695131369, va score: 0.9774133663366336


kfold: 8
Epoch 1/50
5s - loss: 0.6570 - acc: 0.6141 - val_loss: 0.6432 - val_acc: 0.6332
Epoch 2/50
4s - loss: 0.6449 - acc: 0.6296 - val_loss: 0.6307 - val_acc: 0.6332
Epoch 3/50
5s - loss: 0.6386 - acc: 0.6304 - val_loss: 0.6182 - val_acc: 0.6332
Epoch 4/50
5s - loss: 0.6239 - acc: 0.6311 - val_loss: 0.5993 - val_acc: 0.6332
Epoch 5/50
5s - loss: 0.6008 - acc: 0.6329 - val_loss: 0.5691 - val_acc: 0.6332
Epoch 6/50
5s - loss: 0.5794 - acc: 0.6586 - val_loss: 0.5345 - val_acc: 0.6987
Epoch 7/50
5s - loss: 0.5633 - acc: 0.6773 - val_loss: 0.5280 - val_acc: 0.8035
Epoch 8/50
4s - loss: 0.5393 - acc: 0.7231 - val_loss: 0.7618 - val_acc: 0.3668
Epoch 9/50
5s - loss: 0.6568 - acc: 0.5946 - val_loss: 0.6209 - val_acc: 0.6419
Epoch 10/50
4s - loss: 0.6145 - acc: 0.6397 - val_loss: 0.5759 - val_acc: 0.6507
Epoch 11/50
4s - loss: 0.5824 - acc: 0.6513 - val_loss: 0.5754 - val_acc: 0.7686
Epoch 12/50
4s - loss: 0.5514 - acc: 0.6998 - va

tr score: 0.9705745451601805, va score: 0.984731543624161




In [46]:
va_score_per_model

{0: 0.96911011523687585,
 1: 0.96760710553814011,
 2: 0.97052648578811362,
 3: 0.97368644427467954,
 4: 0.97462249980235582,
 5: 0.97120698876805134,
 6: 0.96714783474365351,
 7: 0.97741336633663356,
 8: 0.97742200328407225,
 9: 0.98473154362416104}

In [78]:
# this is actually using the final weights per epoch and we want the best weights for each epoch so
# lets load them for each and calculate area under roc curve
%cd $DATA_DIR
g = glob.glob('model_jun21_kfold_*.model')

model = make_model((128,128,3), grab_optimizer('sgd', 0.005))
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(train_img, train_label, test_size=0.20)

for f in g:
    model.load_weights(f)
    tr_score = score_func(y_train, model.predict(x_train)[:, 0])
    va_score = score_func(y_valid, model.predict(x_valid)[:, 0])
    print('model: {}'.format(f))
    print('tr score: {}, va score: {}'.format(tr_score, va_score))
    print('\n')

/scratch/yns207/data_invasive
model: model_jun21_kfold_6.model
tr score: 0.9761675887842398, va score: 0.98187267465895


model: model_jun21_kfold_0.model
tr score: 0.968041864180145, va score: 0.9701529557668459


model: model_jun21_kfold_7.model
tr score: 0.9741891735735538, va score: 0.9812319140140554


model: model_jun21_kfold_2.model
tr score: 0.9708444074214698, va score: 0.9794336502687061


model: model_jun21_kfold_1.model
tr score: 0.9722183774201617, va score: 0.977904092600248


model: model_jun21_kfold_9.model
tr score: 0.9712507571438186, va score: 0.9769946258784622


model: model_jun21_kfold_5.model
tr score: 0.9702094859802997, va score: 0.9751756924348904


model: model_jun21_kfold_3.model
tr score: 0.9689396430979593, va score: 0.97474162877222


model: model_jun21_kfold_4.model
tr score: 0.9743707611057283, va score: 0.981417941298057


model: model_jun21_kfold_8.model
tr score: 0.9715009161916396, va score: 0.977035965274907




# summary

it looks like model 6 did the best here so we should try to use model 6 to mak ea predictions on the test set or maybe one of the other onef like model 4 or model 9 provide lower validation area under roc but will generalize better.

In [79]:
# woth leaving a note:
# prediction 1,2,3 from jun21 had misalgined the test names
# and the predictions probably explains why my scores were so low
# i realigned them now by no logner reading the file names from the flder
# instaed using the test_set (because thats how the data was produced for
# the test data)
# also iaccidentally overwrote the 3rd submission so 3 and 4 are the same now...
# ugh
%cd $DATA_DIR
batch_size=64
model = make_model((128,128,3), grab_optimizer('sgd', 0.005))
model.load_weights('model_jun21_kfold_6.model')
preds = model.predict(test_img).flatten()

/scratch/yns207/data_invasive


In [80]:
preds[:10]

array([ 0.99625105,  0.11533546,  0.15051772,  0.10293393,  0.99481022,
        0.10451093,  0.0870719 ,  0.99999034,  1.        ,  0.1845029 ], dtype=float32)

In [89]:
print(test_set.shape, preds.shape)

(1531, 2) (1531,)


In [91]:
subm = test_set.copy()

In [93]:
subm['invasive'] = preds
subm.head()

,name,invasive
0,1,0.996251
1,2,0.115335
2,3,0.150518
3,4,0.102934
4,5,0.994810


In [95]:
subm.to_csv(os.path.join(DATA_DIR, 'results', 'subm_june_21_2017_4.gz'), index=False, compression='gzip')

In [96]:
!touch $DATA_DIR/results/*
%ls -lah $DATA_DIR/results

total 332K
drwxr-x--- 2 yns207 yns207 4,0K 21 juin  15:01 ./
drwxr-x--- 5 yns207 yns207 4,0K 21 juin  14:18 ../
-rw-r----- 1 yns207 yns207  17K 21 juin  15:02 subm_june_1_2017_2.gz
-rw-r----- 1 yns207 yns207  19K 21 juin  15:02 subm_june_1_2017_3.gz
-rw-r----- 1 yns207 yns207  19K 21 juin  15:02 subm_june_1_2017_4.gz
-rw-r----- 1 yns207 yns207  19K 21 juin  15:02 subm_june_1_2017_5.gz
-rw-r----- 1 yns207 yns207  14K 21 juin  15:02 subm_june_1_2017_6.gz
-rw-r----- 1 yns207 yns207  17K 21 juin  15:02 subm_june_1_2017.gz
-rw-r----- 1 yns207 yns207 3,4K 21 juin  15:02 subm_june_21_2017_2.gz
-rw-r----- 1 yns207 yns207  17K 21 juin  15:02 subm_june_21_2017_3.gz
-rw-r----- 1 yns207 yns207  17K 21 juin  15:02 subm_june_21_2017_4.gz
-rw-r----- 1 yns207 yns207  17K 21 juin  15:02 subm_june_21_2017.gz
-rw-r----- 1 yns207 yns207  14K 21 juin  15:02 subm_june_2_2017.gz
-rw-r----- 1 yns207 yns207 3,4K 21 juin  15:02 subm_june_8_2017_2.gz
-rw-r----- 1 yns207 yns207  15K 21 juin  15:02 subm_june_8_201